In [ ]:
gsharebits = 0b11
gsharesize = 4
class Instruction:
    def __init__(self, instruction):
        self.instructionVal = bin(instruction)
        self.opcode = instruction >> 26 & 0b111111
        self.rs = instruction >> 21 & 0b11111
        self.rt = instruction >> 16 & 0b11111
        self.rd = instruction >> 11 & 0b11111
        self.shamt = instruction >> 6 & 0b11111
        self.funct = instruction & 0b111111
        self.imm = instruction & 0xFFFF

In [ ]:
class ROB:
    def __init__(self, register, data):
        self.done = 0
        self.data = data
        self.register = register


In [28]:
class ProcessorState:
    def __init__(self):
        self.cycle = 0
        self.registers = [0] *32
        self.issuePointer = 0
        self.commitPointer = 0
        self.RSALU = {# the RS is a dictionary with the key being the reservation station number and the value being an instance
        }
        self.RSALU1 = {}
        self.RSMEM = {}
        self.histReg = 0
        self.pduStates = [0b11]*gsharesize
        self.instructionQ = [] #queues the instructions to hold them for the RS
        self.ROB = {}#the ROB is a dictionary where the index is the key and the value is an ROB class instance
        self.RAT = {}#RAT simply has two possible values, if 0, then the key indexes the RF. if a value, the value indexes the ROB
        self.units = {#simulate execution units. for example, if we want to have 2 ALUS, we just append two instructions in the ALU array. And then then instructions are handled
            "ALU":[],
            "Load/Store": []
        }
        self.memory = [0] * 1024

In [24]:
class PipelineRegister:
    def __init__(self):
        self.flush = 0
        self.instruction = 0
        self.instruction1 = 0
        self.instructionVal = 0
        self.instructionVal1 = 0
        self.prediction = 0
        self.stateIndex = 0
        self.last = 0
        self.data = 0 # any data, depending on the stage. signals are interpreted directly w/o a CU so no need for multiple signals 
        self.done = 0
        self.flush = 0
        self.thisPC = 0 
        self.nextPC = 0

In [30]:
class RS:
    def __init__(self):
        self.tag1 = 0
        self.tag2 = 0
        self.op1 = 0 
        self.op2 = 0
        self.ready = 0

In [ ]:
IF_ID = PipelineRegister()
ID_ISSUE = PipelineRegister()
ISSUE_EXECUTE = PipelineRegister()
EX_MEM = PipelineRegister()
MEM_WB = PipelineRegister()
state = ProcessorState()
state.memory[0] = 0x5
state.memory[1] = 0x7
state.memory[2] = 0x2
state.memory[3] = 0xF
state.memory[4] = 0xA
state.memory[5] = 0x10
state.memory[6] = 0x30
state.memory[7] = 0x1
state.memory[8] = 0xFF
state.memory[9] = 0x55
opcodes = {
    0x0 : 'rtype', 0x8: 'addi', 0xd :'ori', 0xe: 'xori', 0xc:'andi', 0xa:'slti',
    0x23: 'lw', 0x2b : 'sw', 0x4:'beq', 0x2:'j', 0x3 : 'jal', 0x5:'bne'
      
}
notWrites = [0x2b, 0x4, 0x2, 0x5]
arithmetic = [0x0, 0x8, 0xd, 0xe, 0xc, 0xa ]
rtypes = {
    0x20: 'add', 0x22:'sub', 0x24:'and', 0x25: 'or', 0x2a:'slt', 0x26: 'xor', 
    0x27:'nor', 0x0:'sll', 0x2:'srl', 0x8:'jr'
    
}
def signed(data):
        if data & 0x8000:
            return data - (1 << 16)  
        else: return data

In [21]:
instructions = []
with open("imem.txt", 'r') as inst:
    for line in inst:
        parts = line.split(":")
        if len(parts)>1:
            instructions.append(int(parts[1].strip().rstrip(";"),2))

In [26]:
def fetch(state, instructions, IF_ID):
    IF_ID.rs = 0
    IF_ID.rt = 0
    IF_ID.last = 0
    if IF_ID.done:
        return
    elif state.ifidFlush:
        IF_ID.instruction = Instruction(0)
        IF_ID.instructionVal = 0
        inst = Instruction(0)
    else:
        if state.pc < len(instructions):
            inst = Instruction(instructions[state.pc])
        else: 
            IF_ID.last = 1
            inst = Instruction(0)
        IF_ID.thisPC = state.pc
        IF_ID.instruction = inst
        IF_ID.instructionVal = inst.instructionVal
        if opcodes[inst.opcode] == 'j' or opcodes[inst.opcode] =='jal':
            state.pc = inst.imm &0b1111111111
        elif opcodes[inst.opcode] == 'rtype' and rtypes[inst.funct] == 'jr':
            state.pc = state.registers[inst.rs] & 0b1111111111
        else: 
             state.pc += 1
        if state.pc < len(instructions):
            inst1 = Instruction(instructions[state.pc])
        else: 
            IF_ID.last = 1
            inst1 = Instruction(0)
        IF_ID.instruction1 = inst1
        IF_ID.instructionVal1 = inst.instructionVal1
    state.pc = state.pc +1
    IF_ID.nextpc = state.pc
    IF_ID.done = 1
        

In [ ]:
def decode(state, IF_ID, ID_ISSUE):
    #this stage does absolutely nothing in the CAS as the instructions are decoded on the go. this is only here to mimic a cycle for the decode stage(if we want to add one)
    if not IF_ID.done: #checks if IFID is fetching another instruction or not
        return
    ID_ISSUE = IF_ID
    ID_ISSUE.done = 1 #IDEX cannot decode and change its value until execute stage is done
    IF_ID.done = 0 # IFID is now allowed to fetch another instruction



    
    

In [ ]:
def issue(state, ID_ISSUE, ISSUE_EXECUTE):
    if not ID_ISSUE.done:
        return
    else:
        #This is the stage where the instructions are held in an RS if no resources are available or immediately sent to the execution if available. send to the load/store buffers.
        inst = ID_ISSUE.inst
        inst1 = ID_ISSUE.inst1
        validRt1 = opcodes[inst.opcode] == 'rtype' or opcodes[inst.opcode] =='bne' or opcodes[inst.opcode] == 'beq'
        op1 = inst.rs
        op2= inst.rt if validRt1 else inst.imm
        validRt1_1 = opcodes[inst1.opcode] == 'rtype' or opcodes[inst1.opcode] =='bne' or opcodes[inst1.opcode] == 'beq'
        op1_1 = inst1.rs
        op2_1 = inst1.rt if validRt1_1 else inst1.imm
        if len(state.RSALU == 0):
            robInst = ROB(inst.rd if validRT else inst.rt, 0)
            state.ROB[state.issuePointer] = robInst
            state.issuePointer += 1
            rsInst = RS()
            state.RSALU[0] = rsInst


